In [1]:
from __future__ import annotations
import itertools
import pickle

import numpy as np
import scipy.stats as sstt
# import matplotlib.pyplot as plt

from ztbiocas.datasets import uniboinail as ui

In [2]:
# results file of each experiment
fullpath_results_multi = './results/results_multitrain_replication.pkl'
fullpath_results_calib = './results/results_refit_replication.pkl'

# load all results files

with open(fullpath_results_multi, 'rb') as f:
    results_multi = pickle.load(f)['results']

with open(fullpath_results_calib, 'rb') as f:
    results_exper = pickle.load(f)['results']

In [3]:
NUM_REPETITIONS_MULTI = results_multi['num_repetitions']
NUM_REPETITIONS_EXPER = results_exper['num_repetitions']

FULL_CARTPROD_MULTI = list(  # made a list for reuse, notebook-style
    itertools.product(
        range(NUM_REPETITIONS_MULTI),
        range(ui.NUM_SUBJECTS),
        range(ui.NUM_DAYS),
        range(ui.NUM_POSTURES),  # reference posture
        range(ui.NUM_POSTURES),  # arrival posture
    )
)
FULL_CARTPROD_EXPER = list(  # made a list for reuse, notebook-style
    itertools.product(
        range(NUM_REPETITIONS_EXPER),
        range(ui.NUM_SUBJECTS),
        range(ui.NUM_DAYS),
        range(ui.NUM_POSTURES),  # reference posture
        range(ui.NUM_POSTURES),  # arrival posture
    )
)

reps_sublist_multi = list(range(NUM_REPETITIONS_MULTI))
reps_sublist_exper = list(range(NUM_REPETITIONS_EXPER))
subject_sublist     = [0, 1, 2, 3, 4, 5, 6]
day_sublist         = [0, 1, 2, 3, 4, 5, 6, 7]
ref_posture_sublist = [0, 1, 2, 3]  # reference postures
tgt_posture_sublist = [0, 1, 2, 3]  # arrival postures


CUSTOM_CARTPROD_MULTI = list(  # made a list for reuse, notebook-style
    itertools.product(
        reps_sublist_multi,
        subject_sublist,
        day_sublist,
        tgt_posture_sublist,
    )
)

CUSTOM_CARTPROD_EXPER = list(
    itertools.product(  # made a list for reuse, notebook-style
        reps_sublist_exper,
        subject_sublist,
        day_sublist,
        ref_posture_sublist,
        tgt_posture_sublist,
    )
)

In [4]:
# Format accuracy data

# multi-posture training
acc_multi = np.zeros((NUM_REPETITIONS_MULTI, ui.NUM_SUBJECTS, ui.NUM_DAYS, ui.NUM_POSTURES), dtype=np.float32)

for idx_repetition, idx_subject, idx_day, idx_posture in itertools.product(
    range(NUM_REPETITIONS_MULTI), range(ui.NUM_SUBJECTS), range(ui.NUM_DAYS), range(ui.NUM_POSTURES),
):
    acc_multi[idx_repetition, idx_subject, idx_day, idx_posture] = \
        results_multi['repetition'][idx_repetition]['subject'][idx_subject]['day'][idx_day]['posture'][idx_posture]['validation']['accuracy']

custom_mask_mesh = np.ix_(reps_sublist_multi, subject_sublist, day_sublist, tgt_posture_sublist)
acc_multi = acc_multi[custom_mask_mesh]  # select all dimensions simultaneously


# refit
acc_calib_none = np.zeros((NUM_REPETITIONS_EXPER, ui.NUM_SUBJECTS, ui.NUM_DAYS, ui.NUM_POSTURES, ui.NUM_POSTURES), dtype=np.float32)
acc_calib_pcaon = np.zeros((NUM_REPETITIONS_EXPER, ui.NUM_SUBJECTS, ui.NUM_DAYS, ui.NUM_POSTURES, ui.NUM_POSTURES), dtype=np.float32)

for idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture in CUSTOM_CARTPROD_EXPER:

    # no adaptation
    acc_calib_none[idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture] = \
        results_exper['repetition'][idx_repetition]['subject'][idx_subject]['day'][idx_day]['reference_posture'][idx_ref_posture][
            'target_posture'][idx_tgt_posture]['validation']['frozen']['accuracy']

    # online PCA
    acc_calib_pcaon[idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture] = \
        results_exper['repetition'][idx_repetition]['subject'][idx_subject]['day'][idx_day]['reference_posture'][idx_ref_posture][
            'target_posture'][idx_tgt_posture]['validation']['refit']['accuracy']


# calibration modes: discard the posture-diagonal ones
acc_calib_none_flat = []
acc_calib_pcaon_flat = []

for idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture in CUSTOM_CARTPROD_EXPER:
    if idx_tgt_posture != idx_ref_posture:
        acc_calib_none_flat.append(acc_calib_none[idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture])
        acc_calib_pcaon_flat.append(acc_calib_pcaon[idx_repetition, idx_subject, idx_day, idx_ref_posture, idx_tgt_posture])

acc_calib_none_flat = np.array(acc_calib_none_flat, dtype=np.float32)
acc_calib_pcaon_flat = np.array(acc_calib_pcaon_flat, dtype=np.float32)

In [5]:
# Statistics: median and Median Absolute Deviation (MAD)

# median
acc_multi_med = np.median(acc_multi)
acc_calib_none_med = np.median(acc_calib_none_flat)
acc_calib_pcaon_med = np.median(acc_calib_pcaon_flat)

# mad
acc_multi_mad = sstt.median_abs_deviation(acc_multi, axis=(0, 1, 2, 3))
acc_calib_none_mad = sstt.median_abs_deviation(acc_calib_none_flat)
acc_calib_pcaon_mad = sstt.median_abs_deviation(acc_calib_pcaon_flat)


# Display results
print(
    f"\t\t\tmedian +/- mad\n"
    f"---------------------------------------------------------------------------------------------------------\n"
    f"Multiposture train:\t{acc_multi_med:.4f} +/- {acc_multi_mad:.4f}\n"
    f"---------------------------------------------------------------------------------------------------------\n"
    f"No adaptation:\t\t{acc_calib_none_med:.4f} +/- {acc_calib_none_mad:.4f}\n"
    f"Online PCA:\t\t{acc_calib_pcaon_med:.4f} +/- {acc_calib_pcaon_mad:.4f}\n"
)

			median +/- mad
---------------------------------------------------------------------------------------------------------
Multiposture train:	0.9135 +/- 0.0306
---------------------------------------------------------------------------------------------------------
No adaptation:		0.8143 +/- 0.0510
Online PCA:		0.8240 +/- 0.0492



In [6]:
recovered_drop_med_fraction = (acc_calib_pcaon_med - acc_calib_none_med) / (acc_multi_med - acc_calib_none_med)
print(recovered_drop_med_fraction.round(4))

0.0981
